In [43]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [44]:
## same item id is same as movie id, item id column is renamed as movie id
column_names1 = ['user id','movie id','rating','timestamp']
dataset = pd.read_csv('ml-100k/u.data', sep='\t',header=None,names=column_names1)
dataset.head() 

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [45]:
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
column_names2

['movie id',
 'movie title',
 'release date',
 'video release date',
 'IMDb URL',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [46]:
items_dataset = pd.read_csv('ml-100k/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
items_dataset

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
movie_dataset = items_dataset[['movie id','movie title']]
movie_dataset.head()

,movie id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


## Merging required datasets

In [48]:
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='movie id')
merged_dataset.head()

,user id,movie id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [49]:
# Example of a multiple rating scenario by an user to a specific movie:
merged_dataset[(merged_dataset['movie title'] == 'Chasing Amy (1997)') & (merged_dataset['user id'] == 894)]

,user id,movie id,rating,timestamp,movie title
4800,894,246,4,882404137,Chasing Amy (1997)
22340,894,268,3,879896041,Chasing Amy (1997)


In [50]:
refined_dataset = merged_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})
refined_dataset

,user id,movie title,rating
0,1,101 Dalmatians (1996),2.0
1,1,12 Angry Men (1957),5.0
2,1,"20,000 Leagues Under the Sea (1954)",3.0
3,1,2001: A Space Odyssey (1968),4.0
4,1,"Abyss, The (1989)",3.0
...,...,...,...
99688,943,"Wizard of Oz, The (1939)",3.0
99689,943,Wolf (1994),2.0
99690,943,Wyatt Earp (1994),1.0
99691,943,Young Guns (1988),4.0


## Training KNN model to build collaborative recommender

In [51]:
# Reshaping model in such a way that each user has n-dimensional rating space where n is total number of movies
user_to_movie_df = refined_dataset.pivot(
    index='user id',
     columns='movie title',
      values='rating').fillna(0)

user_to_movie_df.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0


In [52]:
# transform matrix to scipy sparse matrix
user_to_movie_sparse_df = csr_matrix(user_to_movie_df.values)
user_to_movie_sparse_df

<943x1664 sparse matrix of type '<class 'numpy.float64'>'
	with 99693 stored elements in Compressed Sparse Row format>

In [53]:
# Fitting KNN model to scipy sparse matrix

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_to_movie_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [54]:
## function to find top n similar users of the given input user 
def get_similar_users(user, n = 5):
  ## input to this function is the user and number of top similar users you want.

  knn_input = np.asarray([user_to_movie_df.values[user-1]])  #.reshape(1,-1)
  # knn_input = user_to_movie_df.iloc[0,:].values.reshape(1,-1)
  distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
  
  print("Top",n,"users who are very much similar to the User-",user, "are: ")
  print(" ")
  for i in range(1,len(distances[0])):
    print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
  return indices.flatten()[1:] + 1, distances.flatten()[1:]

In [55]:
# Specify User id and Number of similar users we want to consider here

user_id = 778
print(" Few of movies seen by the User:")
print(list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])[:10])
similar_user_list, distance_list = get_similar_users(user_id,5)

 Few of movies seen by the User:
['Amityville Horror, The (1979)', 'Angels in the Outfield (1994)', 'Apocalypse Now (1979)', 'Apollo 13 (1995)', 'Austin Powers: International Man of Mystery (1997)', 'Babe (1995)', 'Back to the Future (1985)', 'Blues Brothers, The (1980)', 'Chasing Amy (1997)', 'Clerks (1994)']
Top 5 users who are very much similar to the User- 778 are: 
 
1 . User: 124 separated by distance of 0.4586649429539592
2 . User: 933 separated by distance of 0.5581959868865324
3 . User: 56 separated by distance of 0.5858413112292744
4 . User: 738 separated by distance of 0.5916272517988691
5 . User: 653 separated by distance of 0.5991479757406326


In [56]:
similar_user_list, distance_list

(array([124, 933,  56, 738, 653], dtype=int64),
 array([0.45866494, 0.55819599, 0.58584131, 0.59162725, 0.59914798]))

In [57]:
weightage_list = distance_list/np.sum(distance_list)
weightage_list

array([0.16419139, 0.19982119, 0.20971757, 0.2117888 , 0.21448105])

In [58]:
# Getting ratings of all movies by derived similar users

mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
mov_rtngs_sim_users

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
movies_list = user_to_movie_df.columns
weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
new_rating_matrix = weightage_list*mov_rtngs_sim_users
mean_rating_list = new_rating_matrix.sum(axis =0)

In [60]:

def recommend_movies(n):
  n = min(len(mean_rating_list),n)
  print(list(movies_list[np.argsort(mean_rating_list)[::-1][:n]]))

In [61]:
print("Movies recommended based on similar users are: ")
recommend_movies(10)

Movies recommended based on similar users are: 
['Star Wars (1977)', 'Terminator, The (1984)', "Schindler's List (1993)", 'Fugitive, The (1993)', 'Forrest Gump (1994)', 'Princess Bride, The (1987)', 'Empire Strikes Back, The (1980)', 'Pulp Fiction (1994)', 'Die Hard (1988)', 'Monty Python and the Holy Grail (1974)']


In [62]:
# filter away movies that have been seen by the current user and not seen at all by any user

def filtered_movie_recommendations(n):
  
  first_zero_index = np.where(mean_rating_list == 0)[0][-1]
  sortd_index = np.argsort(mean_rating_list)[::-1]
  sortd_index = sortd_index[:list(sortd_index).index(first_zero_index)]
  n = min(len(sortd_index),n)
  movies_watched = list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])
  filtered_movie_list = list(movies_list[sortd_index])
  count = 0
  final_movie_list = []
  for i in filtered_movie_list:
    if i not in movies_watched:
      count+=1
      final_movie_list.append(i)
    if count == n:
      break
  if count == 0:
    print("There are no movies left which are not seen by the input users and seen by similar users. May be increasing the number of similar users who are to be considered may give a chance of suggesting an unseen good movie.")
  else:
    print(final_movie_list)

In [63]:
filtered_movie_recommendations(10)

['Star Wars (1977)', "Schindler's List (1993)", 'Princess Bride, The (1987)', 'Empire Strikes Back, The (1980)', 'Return of the Jedi (1983)', 'Fargo (1996)', 'Dances with Wolves (1990)', 'Toy Story (1995)', 'Braveheart (1995)', 'Star Trek: First Contact (1996)']


In [64]:
# Giving Input as User id, Number of similar Users to be considered, Number of top movie we want to recommend

def recommender_system(user_id, n_similar_users, n_movies): #, user_to_movie_df, knn_model):
  
  print("Movie seen by the User:")
  print(list(refined_dataset[refined_dataset['user id'] == user_id]['movie title']))
  print("")

  # def get_similar_users(user, user_to_movie_df, knn_model, n = 5):
  def get_similar_users(user, n = 5):
    
    knn_input = np.asarray([user_to_movie_df.values[user-1]])
    
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
    
    print("Top",n,"users who are very much similar to the User-",user, "are: ")
    print(" ")

    for i in range(1,len(distances[0])):
      print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
    print("")
    return indices.flatten()[1:] + 1, distances.flatten()[1:]


  def filtered_movie_recommendations(n = 10):
  
    first_zero_index = np.where(mean_rating_list == 0)[0][-1]
    sortd_index = np.argsort(mean_rating_list)[::-1]
    sortd_index = sortd_index[:list(sortd_index).index(first_zero_index)]
    n = min(len(sortd_index),n)
    movies_watched = list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])
    filtered_movie_list = list(movies_list[sortd_index])
    count = 0
    final_movie_list = []
    for i in filtered_movie_list:
      if i not in movies_watched:
        count+=1
        final_movie_list.append(i)
      if count == n:
        break
    if count == 0:
      print("There are no movies left which are not seen by the input users and seen by similar users. May be increasing the number of similar users who are to be considered may give a chance of suggesting an unseen good movie.")
    else:
      print(final_movie_list)

  similar_user_list, distance_list = get_similar_users(user_id,n_similar_users)
  weightage_list = distance_list/np.sum(distance_list)
  mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
  movies_list = user_to_movie_df.columns
  weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
  new_rating_matrix = weightage_list*mov_rtngs_sim_users
  mean_rating_list = new_rating_matrix.sum(axis =0)
  print("")
  print("Movies recommended based on similar users are: ")
  print("")
  filtered_movie_recommendations(n_movies)

In [65]:
# print("Enter user id")
# user_id= int(input())
# print("number of similar users to be considered")
# sim_users = int(input())
# print("Enter number of movies to be recommended:")
# n_movies = int(input())
# recommender_system(user_id,sim_users,n_movies)
recommender_system(300, 15,15)

Movie seen by the User:
['Air Bud (1997)', 'Air Force One (1997)', 'Beverly Hills Ninja (1997)', 'Booty Call (1997)', 'Bulletproof (1996)', 'Conspiracy Theory (1997)', 'Fargo (1996)', 'Jack (1996)', 'Jungle2Jungle (1997)', 'Liar Liar (1997)', 'Love Jones (1997)', "McHale's Navy (1997)", 'Men in Black (1997)', 'Mimic (1997)', 'Money Talks (1997)', 'Murder at 1600 (1997)', 'Private Parts (1997)', 'Scream (1996)', 'Thin Line Between Love and Hate, A (1996)']

Top 15 users who are very much similar to the User- 300 are: 
 
1 . User: 725 separated by distance of 0.6739742240262109
2 . User: 166 separated by distance of 0.6824285339680983
3 . User: 510 separated by distance of 0.6833028675763237
4 . User: 812 separated by distance of 0.6872706596338897
5 . User: 451 separated by distance of 0.6922027573610239
6 . User: 170 separated by distance of 0.6947477207271661
7 . User: 816 separated by distance of 0.6966898099099204
8 . User: 783 separated by distance of 0.7033756281014425
9 . User: 5

,user id,movie title,rating
0,-1,Air Bud (1997),4.5
1,-1,Air Force One (1997),4.5
2,-1,Beverly Hills Ninja (1997),4.5
3,-1,Booty Call (1997),4.5
4,-1,Bulletproof (1996),4.5


In [125]:
# Giving Input as User id, Number of similar Users to be considered, Number of top movie we want to recommend

def new_recommender_system(user_df, n_similar_users, n_movies): #, user_to_movie_df, knn_model):
  
  print("Movie seen by the User:")
  print(list(user_df["movie title"]))
  print("")
  user_id = -1

  # def get_similar_users(user, user_to_movie_df, knn_model, n = 5):
  def get_similar_users(n = 5):

    knn_input_array = np.array([4.5 if col in movies else 0 for col in user_to_movie_df.columns])
    
    knn_input = np.asarray([knn_input_array])
    
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
    
    print("Top",n,"users who are very much similar to the User-",user_id, "are: ")
    print(" ")

    for i in range(1,len(distances[0])):
      print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
    print("")
    return indices.flatten()[1:] + 1, distances.flatten()[1:]


  def filtered_movie_recommendations(n = 10):
  
    first_zero_index = np.where(mean_rating_list == 0)[0][-1]
    sortd_index = np.argsort(mean_rating_list)[::-1]
    sortd_index = sortd_index[:list(sortd_index).index(first_zero_index)]
    n = min(len(sortd_index),n)
    # movies_watched = list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])
    movies_watched = list(user_df["movie title"])
    filtered_movie_list = list(movies_list[sortd_index])
    count = 0
    final_movie_list = []
    for i in filtered_movie_list:
      if i not in movies_watched:
        count+=1
        final_movie_list.append(i)
      if count == n:
        break
    if count == 0:
      print("There are no movies left which are not seen by the input users and seen by similar users. May be increasing the number of similar users who are to be considered may give a chance of suggesting an unseen good movie.")
    else:
      print(final_movie_list)

  similar_user_list, distance_list = get_similar_users(n_similar_users)
  weightage_list = distance_list/np.sum(distance_list)
  mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
  movies_list = user_to_movie_df.columns
  weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
  new_rating_matrix = weightage_list*mov_rtngs_sim_users
  mean_rating_list = new_rating_matrix.sum(axis =0)
  print("")
  print("Movies recommended based on similar users are: ")
  print("")
  filtered_movie_recommendations(n_movies)

In [129]:
## create a new user row from movies they like

movies = ['Air Bud (1997)', 'Air Force One (1997)', 'Beverly Hills Ninja (1997)', 'Booty Call (1997)', 'Bulletproof (1996)', 'Conspiracy Theory (1997)', 'Fargo (1996)', 'Jack (1996)', 'Jungle2Jungle (1997)', 'Liar Liar (1997)', 'Love Jones (1997)', "McHale's Navy (1997)", 'Men in Black (1997)', 'Mimic (1997)', 'Money Talks (1997)', 'Murder at 1600 (1997)', 'Private Parts (1997)', 'Scream (1996)', 'Thin Line Between Love and Hate, A (1996)']

def new_user_from_movies(movies):
    user_id_list = [-1] * len(movies)
    rating_list = [4.5] * len(movies)
    keys = refined_dataset.columns
    values = [user_id_list, movies, rating_list]
    # dic = {key: value for key in keys for value in values}
    dic = {}
    for col_index in range(len(keys)):
        dic[keys[col_index]] = values[col_index]
    return pd.DataFrame(dic)

new_user_dataaset = new_user_from_movies(movies)
new_user_dataaset.head()
# type(new_user_dataaset["movie title"][0])

new_recommender_system(new_user_dataaset, 15,15)

Movie seen by the User:
['Air Bud (1997)', 'Air Force One (1997)', 'Beverly Hills Ninja (1997)', 'Booty Call (1997)', 'Bulletproof (1996)', 'Conspiracy Theory (1997)', 'Fargo (1996)', 'Jack (1996)', 'Jungle2Jungle (1997)', 'Liar Liar (1997)', 'Love Jones (1997)', "McHale's Navy (1997)", 'Men in Black (1997)', 'Mimic (1997)', 'Money Talks (1997)', 'Murder at 1600 (1997)', 'Private Parts (1997)', 'Scream (1996)', 'Thin Line Between Love and Hate, A (1996)']

Top 15 users who are very much similar to the User- -1 are: 
 
1 . User: 816 separated by distance of 0.6380144444798441
2 . User: 725 separated by distance of 0.6600447046179149
3 . User: 166 separated by distance of 0.6637852876646536
4 . User: 451 separated by distance of 0.6702995876849924
5 . User: 510 separated by distance of 0.6733890191542329
6 . User: 170 separated by distance of 0.6803790247298763
7 . User: 812 separated by distance of 0.6812312392058462
8 . User: 832 separated by distance of 0.6850551105339069
9 . User: 78